# Exchange Rate Data Cleaning

In [1]:
from IPython.display import display
from ipywidgets import FileUpload

upload = FileUpload()
display(upload)

FileUpload(value={}, description='Upload')

In [8]:
import re
import pandas as pd
import codecs
import matplotlib.pyplot as plt
from datetime import datetime

# Get the uploaded file content
uploaded_file_content = list(upload.value.values())[0]['content'].decode('utf-8')

# Extract relevant data
match = re.search(r'\{\\rtf(.+?)\}', uploaded_file_content, re.DOTALL)
if match:
    rtf_data = match.group(1)

# Organize data into a tabular structure
data_rows = re.findall(r'\\cell (.+?)\\row', rtf_data, re.DOTALL)
table_data = []

for row in data_rows:
    cell_data = re.findall(r'\\cell (.*?)\\cell', row, re.DOTALL)
    table_data.append(cell_data)

# Convert tabular structure to Pandas DataFrame
columns = ['period', 'currency_id', 'currency_name_th', 'currency_name_eng', 'buying_sight', 'buying_transfer', 'selling', 'mid_rate']
df = pd.DataFrame(table_data, columns=columns)

df.head()


,period,currency_id,currency_name_th,currency_name_eng,buying_sight,buying_transfer,selling,mid_rate


In [9]:
uploaded_file_content

"{\\rtf1\\ansi\\ansicpg1252\\cocoartf2709\n\\cocoatextscaling0\\cocoaplatform0{\\fonttbl\\f0\\fnil\\fcharset0 Menlo-Regular;}\n{\\colortbl;\\red255\\green255\\blue255;\\red255\\green255\\blue255;\\red0\\green0\\blue0;}\n{\\*\\expandedcolortbl;;\\cssrgb\\c100000\\c100000\\c100000;\\cssrgb\\c0\\c0\\c0;}\n\\paperw11900\\paperh16840\\margl1440\\margr1440\\vieww11520\\viewh8400\\viewkind0\n\\deftab720\n\\pard\\pardeftab720\\partightenfactor0\n\n\\f0\\fs28 \\cf0 \\cb2 \\expnd0\\expndtw0\\kerning0\n\\{'result': \\{'timestamp': '2023-11-17 23:00:41', 'api': 'Daily Weighted-average Interbank Exchange Rate - THB / USD', 'data': \\{'data_header': \\{'report_name_eng': 'Rates of Exchange of Commercial Banks in Bangkok Metropolis (2002-present)', 'report_name_th': '\\uc0\\u3629 \\u3633 \\u3605 \\u3619 \\u3634 \\u3649 \\u3621 \\u3585 \\u3648 \\u3611 \\u3621 \\u3637 \\u3656 \\u3618 \\u3609 \\u3648 \\u3593 \\u3621 \\u3637 \\u3656 \\u3618 \\u3586 \\u3629 \\u3591 \\u3608 \\u3609 \\u3634 \\u3588 \\u3634 

In [11]:
# Expression pattern to extract the required information
pattern = r"'period': '(.*?)', 'currency_id': '(.*?)', 'buying_transfer': '(.*?)', 'selling': '(.*?)'"

# Find all matches using the pattern
matches = re.findall(pattern, uploaded_file_content)

# Organize the extracted information into columns
periods, currency_ids, buying_rates, selling_rates = zip(*matches)

# Clean up the 'Currency ID' column
clean_currency_ids = [re.search(r": '(.*?)'", id_str).group(1) for id_str in currency_ids]

# Create a DataFrame using pandas
data = {
    'Period': periods,
    'Buying Rate': buying_rates,
    'Selling Rate': selling_rates
}

df = pd.DataFrame(data)

# Display the DataFrame
df.head()

,Period,Buying Rate,Selling Rate
0,2019-01-31,31.0788000,31.4128000
1,2019-01-30,31.2991000,31.6240000
2,2019-01-29,31.3881000,31.7288000
3,2019-01-28,31.3155000,31.6586000
4,2019-01-25,31.5261000,31.8527000


In [5]:
df['Period'] = pd.to_datetime(df['Period'])  

# Create a DataFrame with the complete date range
complete_date_range = pd.date_range(start=df['Period'].min(), end=df['Period'].max(), freq='D')
complete_df = pd.DataFrame({'Period': complete_date_range})

# Merge the complete DataFrame with your existing DataFrame
merged_df = pd.merge(complete_df, df, on='Period', how='left')

# Sort the DataFrame based on the 'Period' column
merged_df = merged_df.sort_values('Period')

# Fill missing values with the value before it (Forward filling)
filled_df = merged_df.ffill()

# Display the DataFrame with all missing dates filled
filled_df

,Period,Buying Rate,Selling Rate
0,2019-01-02,32.1983000,32.5345000
1,2019-01-03,32.0442000,32.3939000
2,2019-01-04,31.9008000,32.2298000
3,2019-01-05,31.9008000,32.2298000
4,2019-01-06,31.9008000,32.2298000
...,...,...,...
1482,2023-01-23,32.5272000,32.8542000
1483,2023-01-24,32.5929000,32.9200000
1484,2023-01-25,32.6384000,32.9651000
1485,2023-01-26,32.5341000,32.8627000


In [6]:
filled_df.drop(filled_df[filled_df['Period'] >= '2023-01-01'].index, inplace=True)
filled_df

,Period,Buying Rate,Selling Rate
0,2019-01-02,32.1983000,32.5345000
1,2019-01-03,32.0442000,32.3939000
2,2019-01-04,31.9008000,32.2298000
3,2019-01-05,31.9008000,32.2298000
4,2019-01-06,31.9008000,32.2298000
...,...,...,...
1455,2022-12-27,34.4703000,34.7972000
1456,2022-12-28,34.4900000,34.8244000
1457,2022-12-29,34.5076000,34.8339000
1458,2022-12-30,34.3913000,34.7335000


365 x 4 = 1460 days

In [7]:
# filled_df.to_csv('exchange_rate.csv', index=False)